In [ ]:
import sys
sys.path.append('../src/')

%load_ext autoreload
%autoreload 2

# Position validation

The goal of this notebook is to validate positions of vessels of interest using data sources like the Danish AIS and remote sensing imagery.

In [ ]:
import pandas as pd
import geopandas as gpd
from danish_utils import csv_to_parquet
from pathlib import Path
import pyarrow.parquet as pq
import ee
import geemap

In [ ]:
PATH = Path.cwd().parent.joinpath('data')

## Danish AIS

Let's see if we can find our vessels of interest in the Danish AIS data. 

In [ ]:
path_in = Path('/media/psf/Home/Downloads/aisdk-2018-07/')
path_out = Path('../data/voi/danish_ais/')

csv_to_parquet(path_in, path_out)

In [ ]:
dfs = []
for file in PATH.joinpath('voi', 'danish_ais').glob('*.parquet'):
    fp = pq.read_table(source=file,
                       use_threads=True,
                       filters=[('mmsi', '=', '273317810')])
    df = fp.to_pandas()
    dfs.append(df)

df = pd.concat(dfs)
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(x=df.longitude, y=df.latitude), crs=4326)
len(gdf)

In [ ]:
gdf.explore(tiles='CartoDB Positron')

## Visual investigation

In [ ]:
ee.Initialize()

In [ ]:
# Get sentinel data

def mask_s2_clouds(image):
  """Masks clouds in a Sentinel-2 image using the QA band.

  Args:
      image (ee.Image): A Sentinel-2 image.

  Returns:
      ee.Image: A cloud-masked Sentinel-2 image.
  """
  qa = image.select('QA60')

  # Bits 10 and 11 are clouds and cirrus, respectively.
  cloud_bit_mask = 1 << 10
  cirrus_bit_mask = 1 << 11

  # Both flags should be set to zero, indicating clear conditions.
  mask = (
      qa.bitwiseAnd(cloud_bit_mask)
      .eq(0)
      .And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))
  )

  return image.updateMask(mask).divide(10000)


dataset = (
    ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
    .filterDate('2018-07-10', '2018-07-13')
    # Pre-filter to get less cloudy granules.
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))
    .map(mask_s2_clouds)
)

visualization = {
    'min': 0.0,
    'max': 0.3,
    'bands': ['B4', 'B3', 'B2'],
}

m = geemap.Map()
m.set_center(4.36, 53.46,  8)

In [ ]:
# Import Canopus data

file = PATH.joinpath('voi', 'gfw_tracks', 'CANOPUS - 2012-01-01T00_00_00.000Z,2024-12-31T00_00_00.000Z', 'data.csv')

can = pd.read_csv(file)

In [ ]:
can.timestamp = pd.to_datetime(can.timestamp)

In [ ]:
slice = can[(can.timestamp >= '2018-07-11') & (can.timestamp <= '2018-07-17')].copy()
slice.reset_index(drop=True, inplace=True)

In [ ]:
m.split_map(left_layer=)

In [ ]:
m.add_circle_markers_from_xy(
    slice, x="lon", y="lat", radius=1, color="red")

In [ ]:
m